In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense,Dropout,Bidirectional # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np

reading file

In [ ]:
df = pd.read_csv("/content/train_data.csv")
train_label = df.Label
df2 =pd.read_csv("/content/dev_data.csv")
valid_label = df2.Labels

In [ ]:
# retrive train data
pickle_in = open("pubmed_bertrain.pickle", "rb")
clinical_train = pickle.load(pickle_in)
# retrive valid data
pickle_in = open("pubmed_valid.pickle", "rb")
clinical_valid = pickle.load(pickle_in)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Assuming you have your labels stored in Y_train as a 1D array

# Create an instance of OneHotEncoder
encoder = OneHotEncoder()
train_label_reshaped = train_label.values.reshape(-1, 1)
train_label_encoded = encoder.fit_transform(train_label_reshaped)
Ltrain = train_label_encoded.toarray()
encoder = OneHotEncoder()
valid_label_reshaped = valid_label.values.reshape(-1, 1)
valid_label_encoded = encoder.fit_transform(valid_label_reshaped)
Lvalid = valid_label_encoded.toarray()

# Print the encoded labels
print(Ltrain,Lvalid)

In [ ]:
from imblearn.over_sampling import SMOTE

# Assuming 'X' contains the feature data and 'y' contains the corresponding labels

# Apply SMOTE oversampling
smote = SMOTE()
# Assuming X_train and y_train are your training data and label
clinical_train2, Ltrain2= smote.fit_resample(clinical_train, Ltrain)
clinical_valid2, Lvalid2= smote.fit_resample(clinical_valid, Lvalid)

In [ ]:
clinical_train2 = clinical_train2.reshape(-1, 768, 1)
clinical_valid2 = clinical_valid2.reshape(-1, 768, 1)


In [ ]:
from tensorflow.keras import layers

model_bert = Sequential()
model_bert.add(Bidirectional(LSTM(60, return_sequences=True)))
model_bert.add(Dropout(0.3))
model_bert.add(Bidirectional(LSTM(30)))
model_bert.add(Dropout(0.2))
model_bert.add(Dense(10, activation='relu'))
model_bert.add(Dropout(0.4))
model_bert.add(Dense(3, activation='softmax'))

model_bert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model_bert.fit(clinical_train2, Ltrain2, batch_size=100, epochs=10)



Epoch 1/10
111/111 [==============================] - 262s 2s/step - loss: 1.0653 - accuracy: 0.4326
Epoch 2/10
111/111 [==============================] - 250s 2s/step - loss: 1.0134 - accuracy: 0.4841
Epoch 3/10
111/111 [==============================] - 250s 2s/step - loss: 0.9857 - accuracy: 0.5092
Epoch 4/10
111/111 [==============================] - 244s 2s/step - loss: 0.9673 - accuracy: 0.5221
Epoch 5/10
111/111 [==============================] - 249s 2s/step - loss: 0.9587 - accuracy: 0.5338
Epoch 6/10
111/111 [==============================] - 248s 2s/step - loss: 0.9468 - accuracy: 0.5419
Epoch 7/10
111/111 [==============================] - 249s 2s/step - loss: 0.9385 - accuracy: 0.5439
Epoch 8/10
111/111 [==============================] - 249s 2s/step - loss: 0.9365 - accuracy: 0.5476
Epoch 9/10
111/111 [==============================] - 248s 2s/step - loss: 0.9305 - accuracy: 0.5489
Epoch 10/10
111/111 [==============================] - 250s 2s/step - loss: 0.9293 - accura

In [ ]:
# classification report
from sklearn.metrics import classification_report
y_pred = model_bert.predict(clinical_valid2)
y_predN=np.round(y_pred)
print(classification_report(Lvalid2, y_predN))


204/204 [==============================] - 57s 274ms/step
              precision    recall  f1-score   support

           0       0.55      0.02      0.05      2169
           1       0.60      0.27      0.37      2169
           2       0.63      0.64      0.64      2169

   micro avg       0.62      0.31      0.41      6507
   macro avg       0.59      0.31      0.35      6507
weighted avg       0.59      0.31      0.35      6507
 samples avg       0.31      0.31      0.31      6507



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
